# Extracting Benchmark audio features

In [49]:
#!pip install soundfile
#!pip install numpy
#!pip install scipy
#!pip install librosa

In [50]:
import soundfile as sf
import numpy as np
from scipy import signal
import librosa.feature as feat
import matplotlib.pyplot as plt
from obspy import UTCDateTime, read, Trace, Stream
from obspy.signal.trigger import classic_sta_lta,plot_trigger
from tqdm import tqdm

In [51]:
filename = "audio_example.wav"
sig, sr = sf.read(filename)



sos = signal.butter(6, [5000, 100000], 'bandpass', fs=sr, output='sos')
sig = signal.sosfiltfilt(sos, sig)
trace = Trace(sig)
trace.stats.sampling_rate = 256000



iter = 0
T=0.2
fech = 256000
window_length = 1000/fech
step = window_length/2
STA_LTA_MAX=0
for windowed_tr in tqdm(trace.slide(window_length=window_length, step=step)):

    # J'obtiens une nouvelle fenêtre ou je peux calculer mes features


    # Je calcule le STA/LTA

    STA_LTA = classic_sta_lta(windowed_tr,nsta=210,nlta=500)
    if STA_LTA.max() > STA_LTA_MAX:

        windowed_tr_computation = windowed_tr.copy()

        STA_LTA_MAX = STA_LTA.max()
        # Note : ici je ne garde que le STA_LTA max, je peux savoir ou dans ce signal j'ai calculé
        # mon STA/LTA mais pour une premiere approche pas besoin. Cela veux dire aussi que les features
        # que je vais calculer plus tard ne vont pas être calculé au même temps t.
        idx_max = STA_LTA.argmax()
        #time_start = windowed_tr_computation.stats.starttime




rms = feat.rms(y=windowed_tr_computation.data) 
sc = feat.spectral_centroid(y=windowed_tr_computation.data, sr=sr)
sb = feat.spectral_bandwidth(y=windowed_tr_computation.data,sr=sr)
sf = feat.spectral_flatness(y=windowed_tr_computation.data)

features = [np.mean(rms), np.std(rms), np.min(rms), np.max(rms),\
            np.mean(sc), np.std(sc), np.min(sc), np.max(sc),\
            np.mean(sb), np.std(sb), np.min(sb), np.max(sb),\
            np.mean(sf), np.std(sf), np.min(sf), np.max(sf)]

203it [00:00, 1788.32it/s]


In [52]:
sr

256000

In [53]:
print(features)

[0.0007939342, 0.0, 0.0007939342, 0.0007939342, 43921.87282175432, 0.0, 43921.87282175432, 43921.87282175432, 24885.941892045314, 0.0, 24885.941892045314, 24885.941892045314, 0.1373244073723676, 0.0, 0.1373244073723676, 0.1373244073723676]


In [54]:
rms

array([[0.00079393]], dtype=float32)

In [55]:
np.shape(sig)

(51200,)